In [3]:
from typing import Union, List
import numpy as np
import matplotlib.pyplot as plt
import string
import random
import re
import requests
import os
import textwrap

from nltk.corpus import machado, mac_morpho, floresta, genesis
from nltk.text import Text

In [34]:
ls /home/luan/nltk_data/corpora/

floresta/      machado.zip  mac_morpho.zip*  stopwords.zip*  wordnet.zip*
floresta.zip*  mac_morpho/  stopwords/       wordnet/


In [35]:
!ls

cipher.ipynb  cipher-pt.ipynb  marm01.txt  moby_dick.txt


## True mapping

In [30]:
# create substitution cipher 

letters1 = list(string.ascii_lowercase)
letters2 = list(string.ascii_lowercase)

print(letters1)

# shuffle second list
random.shuffle(letters2)

true_mappings = {}
for k,v in zip(letters1, letters2):
    true_mappings[k] = v

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


This true mapping is the one only known, theoretically, by the sender and receiver, not by the intruder

In [31]:
print(true_mappings)

{'a': 'f', 'b': 'z', 'c': 'h', 'd': 'm', 'e': 'g', 'f': 's', 'g': 'l', 'h': 'x', 'i': 'r', 'j': 'y', 'k': 'u', 'l': 'n', 'm': 'a', 'n': 'p', 'o': 'k', 'p': 'd', 'q': 'b', 'r': 'q', 's': 't', 't': 'v', 'u': 'o', 'v': 'i', 'w': 'j', 'x': 'e', 'y': 'w', 'z': 'c'}


## Language Model

In [32]:
# leveraging ord function to get integers from a character to use as index
ord("a"), ord("b"), ord("c")

(97, 98, 99)

In [36]:
# markov matrix to store bigram probabilities
# we initialize with ones to consider "add-one smoothing"
M = np.ones((26,26))

# initial state distribution (unigrams probabilities)
pi = np.zeros(26)

def update_bigrams(ch1, ch2):
    i = ord(ch1) - 97
    j = ord(ch2) - 97
    M[i,j] += 1
    
def update_unigrams(ch):
    i = ord(ch) - 97
    pi[i] += 1
    
# get log-probability of a word/token
def get_word_prob(word):
    
    probs = []
    # first word index
    i = ord(word[0]) - 97
    probs.append(np.log(pi[i]))
    
    # rest of sentence
    for w_previous, w in zip(word, word[1:]):
        i = ord(w_previous) - 97
        j = ord(w) - 97
        probs.append(np.log(M[i,j]))
        
    # find log-probability
    return sum(probs)

# get log-probability of a document, which is a sequence of words
def get_sequence_prob(doc:Union[str, List]):
    
    if type(doc) == str:
        doc = doc.split()
        
    prob = 0
    for word in doc:
        prob += get_word_prob(word)
        
    return prob

## Creating a Language Model from Moby Dick Book

In [40]:
import io
f = io.open('marm01.txt', 'r', encoding='utf-8')

In [42]:
regex = re.compile('[^a-zA-Z]')

for line in open('marm01.txt', 'r', encoding="latin-1"):
    line = line.rstrip()
    if line:
        # replace non-alpha characters with space
        line = regex.sub(' ', line) 
        tokens = line.lower().split()
        
        # update our language model 
        for token in tokens:
            # update first unigram letter
            ch0 = token[0]
            update_unigrams(ch0)
            
            # update bigrams for the other letters
            for ch1 in token[1:]:
                update_bigrams(ch0, ch1)
                ch0 = ch1

# normalize probabilities
pi /= pi.sum()
M /= M.sum(axis=1, keepdims=True)

In [59]:
regex.sub(" ", original_message)

'o Dr  F lix levantou se tarde  abriu a janela e cumprimentou o sol  O dia   estava espl ndido  uma fresca bafagem do mar vinha quebrar um pouco os ardores   do estio  algumas raras nuvenzinhas brancas  finas e transparentes se   destacavam no azul do c u  Chilreavam na ch cara vizinha   casa do doutor   algumas aves afeitas   vida semi urbana  semi silvestre que lhes pode oferecer   uma ch cara nas Laranjeiras  Parecia que toda a natureza colaborava na   inaugura  o do ano  Aqueles para quem a idade j  desfez o vi o dos primeiros   tempos  n o se ter o esquecido do fervor com que esse dia   saudado na meninice   e na adolesc ncia  Tudo nos parece melhor e mais belo     fruto da nossa ilus o       e alegres com vermos o ano que desponta  n o reparamos que ele   tamb m um   passo para a morte   '

In [44]:
M[:2], pi[:1]

(array([[8.15594160e-05, 2.28366365e-02, 2.50387407e-02, 8.18856537e-02,
         1.63118832e-04, 9.62401109e-03, 1.39466601e-02, 8.97153576e-04,
         3.92300791e-02, 8.97153576e-04, 2.44678248e-04, 8.45771144e-02,
         9.69741457e-02, 1.02193948e-01, 1.54962890e-02, 2.74855232e-02,
         2.18579235e-02, 1.54636653e-01, 1.76331457e-01, 3.53967866e-02,
         7.91126336e-03, 6.60631270e-02, 8.15594160e-05, 8.15594160e-05,
         8.15594160e-05, 1.59856455e-02],
        [1.83603757e-01, 8.53970965e-04, 8.53970965e-04, 1.70794193e-03,
         2.16054654e-01, 8.53970965e-04, 8.53970965e-04, 8.53970965e-04,
         1.22971819e-01, 8.53970965e-03, 8.53970965e-04, 1.02476516e-02,
         2.56191289e-03, 1.70794193e-03, 1.46029035e-01, 8.53970965e-04,
         8.53970965e-04, 2.12638770e-01, 3.92826644e-02, 5.12382579e-03,
         3.75747225e-02, 8.53970965e-04, 8.53970965e-04, 8.53970965e-04,
         1.70794193e-03, 8.53970965e-04]]), array([0.11327047]))

## Encoding Messages

In [53]:
### encode a message

# this is a random excerpt from Project Gutenberg's
# The Adventures of Sherlock Holmes, by Arthur Conan Doyle
# https://www.gutenberg.org/ebooks/1661

# original_message = '''Eu preciso da sua ajuda, estou preso em uma prisão no interior
# do estado. ele vieram por toda parte e me encurralaram, agora estou aqui sofrendo de dor.
# Peço que busquem ajuda o mais rápido possível para me salvar.
# '''

original_message = """o Dr. Félix levantou-se tarde, abriu a janela e cumprimentou o sol. O dia
  estava esplêndido; uma fresca bafagem do mar vinha quebrar um pouco os ardores
  do estio; algumas raras nuvenzinhas brancas, finas e transparentes se
  destacavam no azul do céu. Chilreavam na chácara vizinha à casa do doutor
  algumas aves afeitas à vida semi-urbana, semi-silvestre que lhes pode oferecer
  uma chácara nas Laranjeiras. Parecia que toda a natureza colaborava na
  inauguração do ano. Aqueles para quem a idade já desfez o viço dos primeiros
  tempos, não se terão esquecido do fervor com que esse dia é saudado na meninice
  e na adolescência. Tudo nos parece melhor e mais belo, -- fruto da nossa ilusão,
  -- e alegres com vermos o ano que desponta, não reparamos que ele é também um
  passo para a morte.

"""

In [54]:
def encode_msg(msg):
    
    # lowercase everything and remove non-alpha charcaters
    msg = msg.lower()
    msg = regex.sub(" ", msg)
    
    coded_msg = []
    for ch in msg:
        coded_ch = ch
        if ch in true_mappings:
            coded_ch = true_mappings[ch]
        coded_msg.append(coded_ch)
            
    return "".join(coded_msg)

def decode_msg(msg, word_mapping):
    decoded_msg = []
    for ch in msg:
        decoded_ch = ch
        if ch in word_mapping:
            decoded_ch = word_mapping[ch]
        decoded_msg.append(decoded_ch)
            
    return "".join(decoded_msg)

In [55]:
encoded_msg = encode_msg(original_message)
print(encoded_msg)

k mq  s nre ngifpvko tg vfqmg  fzqro f yfpgnf g hoadqragpvko k tkn  k mrf   gtvfif gtdn pmrmk  oaf sqgthf zfsflga mk afq irpxf bogzqfq oa dkohk kt fqmkqgt   mk gtvrk  fnloaft qfqft poigpcrpxft zqfphft  srpft g vqfptdfqgpvgt tg   mgtvfhfifa pk fcon mk h o  hxrnqgfifa pf hx hfqf ircrpxf   hftf mk mkovkq   fnloaft figt fsgrvft   irmf tgar oqzfpf  tgar trnigtvqg bog nxgt dkmg ksgqghgq   oaf hx hfqf pft nfqfpygrqft  dfqghrf bog vkmf f pfvoqgcf hknfzkqfif pf   rpfoloqf  k mk fpk  fbogngt dfqf boga f rmfmg y  mgtsgc k ir k mkt dqragrqkt   vgadkt  p k tg vgq k gtboghrmk mk sgqikq hka bog gttg mrf   tfomfmk pf agprprhg   g pf fmkngth phrf  vomk pkt dfqghg agnxkq g afrt zgnk     sqovk mf pkttf rnot k       g fnglqgt hka igqakt k fpk bog mgtdkpvf  p k qgdfqfakt bog gng   vfaz a oa   dfttk dfqf f akqvg   


## Genetic Evolutionary Algorithm to decode messages

In [56]:
def generate_dna_pool(n=20):
    dna_pool = []
    for _ in range(n):
        dna = list(string.ascii_lowercase)
        random.shuffle(dna)
        dna_pool.append(dna)
    
    return dna_pool

def procriate_offspring(dna_pool, n_children=3):
    
    offspring = []
    for parent in dna_pool:
        for _ in range(n_children):
            copy = parent.copy()
            i = np.random.randint(len(copy))
            j = np.random.randint(len(copy))
            
            # swap characters
            tmp = copy[i]
            copy[i] = copy[j]
            copy[j] = tmp
            
            offspring.append(copy)
            
    return offspring + dna_pool
        

In [57]:
def run_model(n_epochs=100):
    
    n_survivals = 5
    scores = np.zeros(n_epochs)
    best_dna = None
    best_map = None
    best_score = float('-inf')
    
    dna_pool = generate_dna_pool()
    
    for i in range(n_epochs):
        if i > 0:
            dna_pool = procriate_offspring(dna_pool, n_survivals)

        # calculate score for each dna
        dna2score = {}
        for dna in dna_pool:
            # build a map from current dna sequence
            current_map = {}
            for k,v in zip(letters1, dna):
                current_map[k] = v

            # decode using current map    
            decoded_msg = decode_msg(encoded_msg, current_map)
            score = get_sequence_prob(decoded_msg)

            # store this result
            dna2score[''.join(dna)] = score

            # check if this score is better than the best
            if score > best_score:
                best_score = score
                best_dna = dna
                best_map = current_map

        scores[i] = np.mean(list(dna2score.values()))

        # keep the best DNAs, survival of the fittest, using n_survivals
        sorted_dna = sorted(dna2score.items(), key=lambda x: x[1], reverse=True)
        dna_pool = [list(k) for k,v in sorted_dna[:n_survivals]]
#         [list(k) for k, v in sorted_dna[:5]]

        if i % 200 == 0:
            print("iter:", i, "score:", scores[i], "best so far:", best_score)
        
    return best_dna, best_map, best_score, scores

In [58]:
# Run the evolution!
n_epochs = 1000
best_dna, best_map, best_score, scores = run_model(n_epochs)

/home/luan/.local/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log


iter: 0 score: -inf best so far: -inf
iter: 200 score: -inf best so far: -1381.7913519609867
iter: 400 score: -inf best so far: -1381.7913519609867
iter: 600 score: -inf best so far: -1381.7913519609867
iter: 800 score: -inf best so far: -1381.7913519609867


In [60]:
# use best score
decoded_msg = decode_msg(encoded_msg, best_map)

print("LL of decoded with best mapping: ", get_sequence_prob(decoded_msg))
print("LL of original mapping: ", get_sequence_prob(regex.sub(" ", original_message.lower())))

for true, v in true_mappings.items():
    pred = best_map[v] # best map is a reverse map
    if true != pred:
        print(f"true: {true}, pred: {pred}")

LL of decoded with best mapping:  -1381.7913519609867
LL of original mapping:  -1381.7913519609867
true: k, pred: w
true: w, pred: k


In [61]:
print("decoded message:\n\n", textwrap.fill(decoded_msg)) 

print("\noriginal message:\n\n", original_message)

decoded message:

 o dr  f lix levantou se tarde  abriu a janela e cumprimentou o sol  o
dia   estava espl ndido  uma fresca bafagem do mar vinha quebrar um
pouco os ardores   do estio  algumas raras nuvenzinhas brancas  finas
e transparentes se   destacavam no azul do c u  chilreavam na ch cara
vizinha   casa do doutor   algumas aves afeitas   vida semi urbana
semi silvestre que lhes pode oferecer   uma ch cara nas laranjeiras
parecia que toda a natureza colaborava na   inaugura  o do ano
aqueles para quem a idade j  desfez o vi o dos primeiros   tempos  n o
se ter o esquecido do fervor com que esse dia   saudado na meninice
e na adolesc ncia  tudo nos parece melhor e mais belo     fruto da
nossa ilus o       e alegres com vermos o ano que desponta  n o
reparamos que ele   tamb m um   passo para a morte

original message:

 o Dr. Félix levantou-se tarde, abriu a janela e cumprimentou o sol. O dia
  estava esplêndido; uma fresca bafagem do mar vinha quebrar um pouco os ardores
  do est